In [1]:
import pandas as pd

In [10]:
df_cs = pd.read_csv("./data/gold/cross_section.csv", dtype={"mep_id": str})
df_graph_data = pd.read_csv("./data/gold/gephi_graph_data.csv")

In [68]:
# correlation matrix
print(
    df_graph_data[["modularity_class", "degree", "Authority", "eigencentrality"]].corr()
)

                  modularity_class    degree  Authority  eigencentrality
modularity_class          1.000000  0.385273   0.200847         0.208553
degree                    0.385273  1.000000   0.883139         0.899061
Authority                 0.200847  0.883139   1.000000         0.981707
eigencentrality           0.208553  0.899061   0.981707         1.000000


In [83]:
graph_var_to_choose = "eigencentrality"

df_merged = df_cs.merge(
    df_graph_data[["Id", "modularity_class", graph_var_to_choose]],
    left_on="mep_id",
    right_on="Id",
    how="left",
)

# Get dummies for modularity class
df_merged = pd.get_dummies(df_merged, columns=["modularity_class"])

# Drop the reference classes
df_merged = df_merged.drop(
    columns=[
        "modularity_class_0",
        "country_0",
        "political_group_4273",
        "COMMITTEE_PARLIAMENTARY_STANDING", # because it is empty
    ]
)

# Convert everything to float
df_merged = df_merged.astype(float)

In [84]:
from statsmodels.api import OLS


df = df_merged.drop(columns=["mep_id", "Id", "questions_log"])

Y = df["questions"]

X = df.drop(columns=["questions"])

X['const'] = 1

# Do a simlpe regression
model = OLS(Y, X)

results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              questions   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     10.28
Date:                Fri, 02 May 2025   Prob (F-statistic):           1.25e-94
Time:                        21:28:31   Log-Likelihood:                -7749.4
No. Observations:                1353   AIC:                         1.567e+04
Df Residuals:                    1267   BIC:                         1.612e+04
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
meetings                             -0.0009      0.052     -0.017      0.987      -0.103       0.102
country_AUT                          -9.2004     15.323     -0.600      0.548     -39.262      20.861
country_BEL                          57.0382     14.421      3.955      0.000      28.747      85.330
country_BGR                           0.4379     16.887      0.026      0.979     -32.693      33.568
country_CYP                          45.3526     27.279      1.663      0.097      -8.164      98.869
country_CZE                         -20.8806     14.432     -1.447      0.148     -49.194       7.433
country_DEU                         -22.1972     10.078     -2.203      0.028     -41.969      -2.426
country_DNK                          12.8948     17.575      0.734      0.463     -21.585      47.375
country_ESP                          66.8004     10.580      6.314      0.000      46.044      87.557
country_EST                         -45.7960     23.908     -1.916      0.056     -92.699       1.107
country_FIN                         -28.3243     22.203     -1.276      0.202     -71.883      15.234
country_GBR                          -8.6619     12.364     -0.701      0.484     -32.919      15.595
country_GRC                          73.6473     15.364      4.793      0.000      43.505     103.789
country_HRV                          29.6219     21.088      1.405      0.160     -11.750      70.994
country_HUN                           5.1514     16.477      0.313      0.755     -27.173      37.476
country_IRL                          43.6814     19.840      2.202      0.028       4.759      82.604
country_ITA                          33.2027     10.038      3.308      0.001      13.509      52.896
country_LTU                          -3.0771     19.501     -0.158      0.875     -41.335      35.181
country_LUX                         -17.0422     24.846     -0.686      0.493     -65.787      31.702
country_LVA                         -50.2329     22.069     -2.276      0.023     -93.529      -6.937
country_MLT                          40.7556     29.102      1.400      0.162     -16.339      97.850
country_NLD                          -0.8944     14.047     -0.064      0.949     -28.453      26.664
country_POL                          -1.9547     11.442     -0.171      0.864     -24.403      20.493
country_PRT                          29.8243     14.029      2.126      0.034       2.301      57.348
country_ROU                          15.1949     12.531      1.213      0.225      -9.388      39.778
country_SVK                          20.2749     15.419      1.315      0.189      -9.975      50.525
country_SVN                          28.2435     22.949      1.231      0.219     -16.780      73.267
country_SWE            

## Testando clogaritimizado

In [85]:
from statsmodels.api import OLS
import numpy as np

df = df_merged.drop(columns=["mep_id", "Id", "questions_log"])

df["questions"] = np.log(df["questions"] + 1)

Y = df["questions"]

X = df.drop(columns=["questions"])


X['const'] = 1

# Do a simlpe regression
model = OLS(Y, X)

results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              questions   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     20.24
Date:                Fri, 02 May 2025   Prob (F-statistic):          4.19e-180
Time:                        21:28:37   Log-Likelihood:                -1801.5
No. Observations:                1353   AIC:                             3775.
Df Residuals:                    1267   BIC:                             4223.
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
meetings                              0.0014      0.001      2.212      0.027       0.000       0.003
country_AUT                          -0.2830      0.189     -1.498      0.134      -0.653       0.088
country_BEL                           0.3308      0.178      1.861      0.063      -0.018       0.679
country_BGR                          -0.2287      0.208     -1.099      0.272      -0.637       0.180
country_CYP                           0.0365      0.336      0.109      0.913      -0.623       0.696
country_CZE                          -0.4385      0.178     -2.465      0.014      -0.787      -0.089
country_DEU                          -0.5498      0.124     -4.426      0.000      -0.794      -0.306
country_DNK                           0.1171      0.217      0.541      0.589      -0.308       0.542
country_ESP                           0.5653      0.130      4.335      0.000       0.309       0.821
country_EST                          -0.9915      0.295     -3.365      0.001      -1.570      -0.413
country_FIN                          -0.7945      0.274     -2.903      0.004      -1.331      -0.258
country_GBR                          -0.3924      0.152     -2.575      0.010      -0.691      -0.093
country_GRC                           0.8349      0.189      4.409      0.000       0.463       1.206
country_HRV                           0.3734      0.260      1.437      0.151      -0.137       0.883
country_HUN                          -0.2304      0.203     -1.135      0.257      -0.629       0.168
country_IRL                           0.2721      0.245      1.113      0.266      -0.208       0.752
country_ITA                           0.3120      0.124      2.522      0.012       0.069       0.555
country_LTU                          -0.3599      0.240     -1.497      0.135      -0.831       0.112
country_LUX                          -0.7053      0.306     -2.303      0.021      -1.306      -0.105
country_LVA                          -1.5433      0.272     -5.674      0.000      -2.077      -1.010
country_MLT                           0.5473      0.359      1.526      0.127      -0.156       1.251
country_NLD                          -0.1756      0.173     -1.014      0.311      -0.515       0.164
country_POL                          -0.2582      0.141     -1.830      0.067      -0.535       0.019
country_PRT                           0.0089      0.173      0.051      0.959      -0.330       0.348
country_ROU                          -0.1363      0.154     -0.882      0.378      -0.439       0.167
country_SVK                           0.2653      0.190      1.396      0.163      -0.108       0.638
country_SVN                           0.5245      0.283      1.854      0.064      -0.030       1.079
country_SWE            

In [88]:
df_graph_data[df_graph_data['modularity_class'] == 9].sort_values(by='eigencentrality', ascending=False)

,Id,Label,timeset,type,modularity_class,indegree,outdegree,degree,weighted indegree,weighted outdegree,weighted degree,Authority,Hub,componentnumber,strongcompnum,stat_inf_class,clustering,eigencentrality
38,197654,NaN,NaN,NaN,9,128,0,128,311,0,311,0.083265,0.000000,0,39,0,0,0.677249
39,96668,Seán KELLY,NaN,Mep,9,85,0,85,142,0,142,0.064284,0.000000,0,38,0,0,0.449735
227,101585,Niels FUGLSANG,NaN,Mep,9,79,0,79,187,0,187,0.063928,0.000000,0,225,0,0,0.417989
490,124872,Morten PETERSEN,NaN,Mep,9,75,0,75,146,0,146,0.055215,0.000000,0,485,0,0,0.396825
365,197573,NaN,NaN,NaN,9,54,0,54,82,0,82,0.047799,0.000000,0,352,0,0,0.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1922,788025840814-90,JAE Public Affairs,NaN,Self-employed individuals,9,0,4,4,0,6,6,0.000000,0.008060,0,1729,0,0,0.000000
1905,818300434979-49,PGE Polska Grupa Energetyczna SA,NaN,Companies & groups,9,0,35,35,0,55,55,0.000000,0.065799,0,1715,0,0,0.000000
1873,43859808000-87,European Association for Storage of Energy,NaN,Trade and business associations,9,0,12,12,0,16,16,0.000000,0.024938,0,1693,0,0,0.000000
1870,024782946888-95,Danfoss A/S,NaN,Companies & groups,9,0,12,12,0,25,25,0.000000,0.027225,0,1690,0,0,0.000000
